In [2]:
"""Importing Library"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('whitegrid')
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn import linear_model, datasets
from sklearn import tree
from IPython.display import Image  
from sklearn.externals.six import StringIO  
import pydotplus

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [94]:
"""Reading Data"""
data1 = pd.read_csv('./month_1.csv')
data1.head()


,customerId,accountId,transactionDate,transactionTime,terminalId,transactionAmount,transactionRemain,state,transactionCode
0,guke_0,zhanghu_51317,"(2015, 3, 21)",160125.0,zhongduan_0,-300.000 CN¥,11393.314 CN¥,D,ma_0
1,guke_0,zhanghu_51317,"(2015, 3, 21)",155406.0,zhongduan_1,-2000.000 CN¥,11693.314 CN¥,D,ma_1
2,guke_0,zhanghu_51317,"(2015, 3, 21)",3519.0,zhongduan_2,-5705.000 CN¥,13693.314 CN¥,D,ma_1
3,guke_0,zhanghu_51317,"(2015, 3, 21)",3442.0,zhongduan_3,-150.000 CN¥,19398.314 CN¥,D,ma_0
4,guke_1,zhanghu_51318,"(2015, 3, 21)",165927.0,zhongduan_4,-2000.000 CN¥,6585.130 CN¥,D,ma_1


In [93]:
"""
Now setting up some changes on data

Example: cleaning!

Function for cleaning the customerId
Function for cleaning the accountIdId
Function for cleaning the transactionCode

"""
def simplify_customerId(df):
    T = df.customerId.apply(lambda x: x[5:])
    df.customerId = T
    return df


def simplify_accountId(df):
    T = df.accountId.apply(lambda x: x[8:])
    df.accountId = T
    return df

def simplify_terminalId(df):
    T = df.terminalId.apply(lambda x: x[10:])
    df.terminalId = T
    return df

def simplify_transactionCode(df):
    T = df.transactionCode.apply(lambda x: x[3:])
    df.transactionCode = T
    return df

def transform_features(df):
    df = simplify_customerId(df)
    df = simplify_accountId(df)
    df = simplify_terminalId(df)
    df = simplify_transactionCode(df)
    return df
    



In [95]:
data1 = transform_features(data1)
data1

,customerId,accountId,transactionDate,transactionTime,terminalId,transactionAmount,transactionRemain,state,transactionCode
0,0,51317,"(2015, 3, 21)",160125.0,0,-300.000 CN¥,11393.314 CN¥,D,0
1,0,51317,"(2015, 3, 21)",155406.0,1,-2000.000 CN¥,11693.314 CN¥,D,1
2,0,51317,"(2015, 3, 21)",3519.0,2,-5705.000 CN¥,13693.314 CN¥,D,1
3,0,51317,"(2015, 3, 21)",3442.0,3,-150.000 CN¥,19398.314 CN¥,D,0
4,1,51318,"(2015, 3, 21)",165927.0,4,-2000.000 CN¥,6585.130 CN¥,D,1
5,2,51319,"(2015, 3, 21)",143155.0,5,-2000.000 CN¥,7201.000 CN¥,D,1
6,3,51320,"(2015, 3, 21)",202233.0,6,-2405.000 CN¥,16548.991 CN¥,D,1
7,3,51320,"(2015, 3, 21)",193954.0,7,-170.000 CN¥,18953.991 CN¥,D,0
8,3,51320,"(2015, 3, 21)",165122.0,8,-1130.000 CN¥,19123.991 CN¥,D,0
9,3,51320,"(2015, 3, 21)",161054.0,9,-600.000 CN¥,20253.991 CN¥,D,0
